# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f02108921f0>
├── 'a' --> tensor([[-1.6570,  0.6017, -0.8354],
│                   [-0.3116, -1.0796,  0.7825]])
└── 'x' --> <FastTreeValue 0x7f0210892340>
    └── 'c' --> tensor([[ 1.2673,  0.6671, -0.6309, -0.2700],
                        [ 0.5626, -1.0116, -1.7509,  0.5799],
                        [ 0.8712,  2.0530,  0.3501,  1.5526]])

In [4]:
t.a

tensor([[-1.6570,  0.6017, -0.8354],
        [-0.3116, -1.0796,  0.7825]])

In [5]:
%timeit t.a

65.9 ns ± 0.36 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f02108921f0>
├── 'a' --> tensor([[ 0.7265,  0.2391,  1.0694],
│                   [-0.5744, -0.8073, -1.5563]])
└── 'x' --> <FastTreeValue 0x7f0210892340>
    └── 'c' --> tensor([[ 1.2673,  0.6671, -0.6309, -0.2700],
                        [ 0.5626, -1.0116, -1.7509,  0.5799],
                        [ 0.8712,  2.0530,  0.3501,  1.5526]])

In [7]:
%timeit t.a = new_value

67.2 ns ± 0.927 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.6570,  0.6017, -0.8354],
               [-0.3116, -1.0796,  0.7825]]),
    x: Batch(
           c: tensor([[ 1.2673,  0.6671, -0.6309, -0.2700],
                      [ 0.5626, -1.0116, -1.7509,  0.5799],
                      [ 0.8712,  2.0530,  0.3501,  1.5526]]),
       ),
)

In [10]:
b.a

tensor([[-1.6570,  0.6017, -0.8354],
        [-0.3116, -1.0796,  0.7825]])

In [11]:
%timeit b.a

64.1 ns ± 0.829 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 2.1637, -0.2761,  0.6698],
               [ 0.9810, -0.9415,  0.5328]]),
    x: Batch(
           c: tensor([[ 1.2673,  0.6671, -0.6309, -0.2700],
                      [ 0.5626, -1.0116, -1.7509,  0.5799],
                      [ 0.8712,  2.0530,  0.3501,  1.5526]]),
       ),
)

In [13]:
%timeit b.a = new_value

582 ns ± 6.08 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

949 ns ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.5 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

272 µs ± 4.73 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

255 µs ± 6.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0210892190>
├── 'a' --> tensor([[[-1.6570,  0.6017, -0.8354],
│                    [-0.3116, -1.0796,  0.7825]],
│           
│                   [[-1.6570,  0.6017, -0.8354],
│                    [-0.3116, -1.0796,  0.7825]],
│           
│                   [[-1.6570,  0.6017, -0.8354],
│                    [-0.3116, -1.0796,  0.7825]],
│           
│                   [[-1.6570,  0.6017, -0.8354],
│                    [-0.3116, -1.0796,  0.7825]],
│           
│                   [[-1.6570,  0.6017, -0.8354],
│                    [-0.3116, -1.0796,  0.7825]],
│           
│                   [[-1.6570,  0.6017, -0.8354],
│                    [-0.3116, -1.0796,  0.7825]],
│           
│                   [[-1.6570,  0.6017, -0.8354],
│                    [-0.3116, -1.0796,  0.7825]],
│           
│                   [[-1.6570,  0.6017, -0.8354],
│                    [-0.3116, -1.0796,  0.7825]]])
└── 'x' --> <FastTreeValue 0x7f016d186100>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

42 µs ± 916 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f016655ceb0>
├── 'a' --> tensor([[-1.6570,  0.6017, -0.8354],
│                   [-0.3116, -1.0796,  0.7825],
│                   [-1.6570,  0.6017, -0.8354],
│                   [-0.3116, -1.0796,  0.7825],
│                   [-1.6570,  0.6017, -0.8354],
│                   [-0.3116, -1.0796,  0.7825],
│                   [-1.6570,  0.6017, -0.8354],
│                   [-0.3116, -1.0796,  0.7825],
│                   [-1.6570,  0.6017, -0.8354],
│                   [-0.3116, -1.0796,  0.7825],
│                   [-1.6570,  0.6017, -0.8354],
│                   [-0.3116, -1.0796,  0.7825],
│                   [-1.6570,  0.6017, -0.8354],
│                   [-0.3116, -1.0796,  0.7825],
│                   [-1.6570,  0.6017, -0.8354],
│                   [-0.3116, -1.0796,  0.7825]])
└── 'x' --> <FastTreeValue 0x7f016655cf70>
    └── 'c' --> tensor([[ 1.2673,  0.6671, -0.6309, -0.2700],
                        [ 0.5626, -1.0116, -1.7509,  0.5799],
                 

In [23]:
%timeit t_cat(trees)

37.6 µs ± 336 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77.3 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.2673,  0.6671, -0.6309, -0.2700],
                       [ 0.5626, -1.0116, -1.7509,  0.5799],
                       [ 0.8712,  2.0530,  0.3501,  1.5526]],
              
                      [[ 1.2673,  0.6671, -0.6309, -0.2700],
                       [ 0.5626, -1.0116, -1.7509,  0.5799],
                       [ 0.8712,  2.0530,  0.3501,  1.5526]],
              
                      [[ 1.2673,  0.6671, -0.6309, -0.2700],
                       [ 0.5626, -1.0116, -1.7509,  0.5799],
                       [ 0.8712,  2.0530,  0.3501,  1.5526]],
              
                      [[ 1.2673,  0.6671, -0.6309, -0.2700],
                       [ 0.5626, -1.0116, -1.7509,  0.5799],
                       [ 0.8712,  2.0530,  0.3501,  1.5526]],
              
                      [[ 1.2673,  0.6671, -0.6309, -0.2700],
                       [ 0.5626, -1.0116, -1.7509,  0.5799],
                       [ 0.8712,  2.0530,  0.3501,  1.5526]],

In [26]:
%timeit Batch.stack(batches)

98.1 µs ± 4.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.2673,  0.6671, -0.6309, -0.2700],
                      [ 0.5626, -1.0116, -1.7509,  0.5799],
                      [ 0.8712,  2.0530,  0.3501,  1.5526],
                      [ 1.2673,  0.6671, -0.6309, -0.2700],
                      [ 0.5626, -1.0116, -1.7509,  0.5799],
                      [ 0.8712,  2.0530,  0.3501,  1.5526],
                      [ 1.2673,  0.6671, -0.6309, -0.2700],
                      [ 0.5626, -1.0116, -1.7509,  0.5799],
                      [ 0.8712,  2.0530,  0.3501,  1.5526],
                      [ 1.2673,  0.6671, -0.6309, -0.2700],
                      [ 0.5626, -1.0116, -1.7509,  0.5799],
                      [ 0.8712,  2.0530,  0.3501,  1.5526],
                      [ 1.2673,  0.6671, -0.6309, -0.2700],
                      [ 0.5626, -1.0116, -1.7509,  0.5799],
                      [ 0.8712,  2.0530,  0.3501,  1.5526],
                      [ 1.2673,  0.6671, -0.6309, -0.2700],
                   

In [28]:
%timeit Batch.cat(batches)

168 µs ± 3.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

546 µs ± 6.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
